In [1]:
import re
import pandas as pd
import numpy as np
# cleaning of html tags
from bs4 import BeautifulSoup
# stopwords
from nltk.corpus import stopwords
# tokenization
# https://pypi.org/project/tokenizers/
from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)

In [3]:
%cd ..

/home/btr-dev/wrkspc/prj/salty-hackers/ML-Model/Saltiest-Hackers-ML-Model


In [4]:
!ls

data  models  notebooks  references  src


In [5]:
DIR = 'data/raw/gcp-bq-full/'

FILES = [ str(f) for f in list(range(1,18))]
DIR+FILES[0]

'data/raw/gcp-bq-full/1'

In [6]:
df = pd.read_csv(DIR+FILES[0])
df.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,NaN,NaN,I started to write a C++ template class that w...,NaN,cpeterso,NaN,1.338489e+09,2012-05-31 18:32:12 UTC,comment,4049595,4049139.0,NaN,NaN,NaN
1,NaN,NaN,&gt; I&#x27;m guessing over $200B of Amazon&#x...,NaN,fweespeech,NaN,1.498437e+09,2017-06-26 00:33:02 UTC,comment,14633082,14632856.0,NaN,NaN,NaN
2,NaN,NaN,I don&#x27;t know what to say - that just soun...,NaN,chrisseaton,NaN,1.552875e+09,2019-03-18 02:17:05 UTC,comment,19418238,19418216.0,NaN,NaN,NaN
3,NaN,NaN,if you&#x27;re going to make the accusation th...,NaN,Aloha,NaN,1.515732e+09,2018-01-12 04:37:28 UTC,comment,16130436,16130413.0,NaN,NaN,NaN
4,NaN,NaN,With the current trend of simplifying your int...,NaN,drill_sarge,NaN,1.382406e+09,2013-10-22 01:40:09 UTC,comment,6589315,6588825.0,NaN,NaN,NaN


## SO after a few days of reading I think I finally got it
I will start by creating a training set. I will loop over the collected data, which is comprised of a shuffled set of all comments from the Hacker News website, process the text and select for a few requirements:
- We should exclude comments with low word counts so that the final model doesn't tune its self to any word in particular
- We should aim for the highest amount of unique words possible
- 

In [7]:
def scrub(doc):
    return re.sub(r'[^A-Za-z\s]', '', str(doc))

def process_text(df):
    # only those comments with not null values
    df = df.loc[df['type'] == 'comment'][['text']]
    df = df.dropna()    
    # clean the text using bs4
    df['text'] = df['text'].apply(lambda x: BeautifulSoup(x).get_text())
    # regex remove all non-letters && to lower
    df['text'] = df['text'].apply(scrub)
    df['text'] = df['text'].str.lower()
    return df


def remove_stops(doc):
    # get word frequencies in document
    word_freq = pd.Series(' '.join(df['text'].str).split()).value_counts()
    # make lists of most common words and words that appear once only
    common = list(word_freq[:10].index)
    rare = list(word_freq[word_freq.values < 2].index)
    # start with NLTK stopwords
    stop_words = list(nltk.corpus.stopwords.words('english'))    
    # add the common and rare words to the set
    stop_words = set(stop_words + common + rare)
    # use regex for stopword removal
    pat = r'\b(?:{})\b'.format('|'.join(stop_words))    
    df['text'] = df['text'].str.replace(pat, '')
    df['text'] = df['text'].str.replace(r'\s+', ' ')
    # retaining comments with 30 or more words
    df = df.loc[df['text'].apply(lambda x: len(str(x).split(" "))).values > 30] 
    
    return df

In [7]:
!ls

data  models  notebooks  references  src


In [8]:
samp = df.sample(100)

In [10]:
df1 = process_text(samp)

In [11]:
df1

,text
506826,is happening with bitcoin and toryou cant cen...
513218,have you actually tried it i dont really under...
38767,no google makes money off connecting ads to pr...
1063988,there was not i appreciate your helpful attitu...
1252538,i took a hybrid course which is online but you...
...,...
589899,dont forget the part where uber is selling the...
316361,dude i get what youre saying but youre like a ...
1235781,combining abbr and conceal could indeed reduce...
524577,as far as i know icbms are still unstoppable o...


In [17]:
BeautifulSoup(df['text'].values[0]).get_text()

'I started to write a C++ template class that would implement strongly-typed ints (so Celsius and Fahrenheit types could behave like ints, but have distinct types).I gave up after this "simple" idea approached 200 lines of code implementing all the operator overloads. I guess the lesson is that primitives are complex, even if you just want to give them a new name. Also, the expression int/int produces an int, but what should the expression FahrenheitInt/FahrenheitInt produce? A unitless int? A FahrenheitInt?'

In [19]:
text = df1['text'].values[0]

In [ ]:
word_freq = pd.Series(' '.join(df1['text'].str).split()).value_counts()

/home/btr-dev/res/miniconda3/envs/salty-model/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [18]:
tokenizer = BertWordPieceTokenizer("data/external/vocab/bert-base-uncased-vocab.txt", lowercase=True)

In [20]:
encoded = tokenizer.encode(text)

In [21]:
encoded.tokens

['[CLS]',
 'is',
 'happening',
 'with',
 'bit',
 '##co',
 '##in',
 'and',
 'tory',
 '##ou',
 'can',
 '##t',
 'central',
 '##ize',
 'bit',
 '##co',
 '##in',
 'that',
 'would',
 'be',
 'like',
 'trying',
 'to',
 'central',
 '##ize',
 'pp',
 'sharing',
 'the',
 'protocol',
 'doesn',
 '##t',
 'allow',
 'it',
 'an',
 'alternative',
 'digital',
 'currency',
 'may',
 'emerge',
 'that',
 'is',
 'centralized',
 'or',
 'more',
 'likely',
 'a',
 'single',
 'currency',
 'exchange',
 'will',
 'dominate',
 'bit',
 '##co',
 '##in',
 'but',
 'in',
 'reality',
 'exchanges',
 'would',
 'become',
 'obsolete',
 'if',
 'bit',
 '##co',
 '##in',
 'really',
 'took',
 'off',
 'you',
 'would',
 'just',
 'get',
 'paid',
 'and',
 'purchase',
 'goods',
 'all',
 'in',
 'bit',
 '##co',
 '##in',
 'no',
 'need',
 'to',
 'exchange',
 'to',
 'another',
 'currency',
 '[SEP]']

In [10]:
text = BeautifulSoup(df['text'][0]).get_text()
text

'I started to write a C++ template class that would implement strongly-typed ints (so Celsius and Fahrenheit types could behave like ints, but have distinct types).I gave up after this "simple" idea approached 200 lines of code implementing all the operator overloads. I guess the lesson is that primitives are complex, even if you just want to give them a new name. Also, the expression int/int produces an int, but what should the expression FahrenheitInt/FahrenheitInt produce? A unitless int? A FahrenheitInt?'

In [13]:
text = re.sub(r'[^A-Za-z\s]', '', df['text'][0]).lower()

In [34]:
regexp_tokenize(text, pattern = '\s', gaps=True)

['i',
 'started',
 'to',
 'write',
 'a',
 'c',
 'template',
 'class',
 'that',
 'would',
 'implement',
 'stronglytyped',
 'ints',
 'so',
 'celsius',
 'and',
 'fahrenheit',
 'types',
 'could',
 'behave',
 'like',
 'ints',
 'but',
 'have',
 'distinct',
 'typespi',
 'gave',
 'up',
 'after',
 'this',
 'simple',
 'idea',
 'approached',
 'lines',
 'of',
 'code',
 'implementing',
 'all',
 'the',
 'operator',
 'overloads',
 'i',
 'guess',
 'the',
 'lesson',
 'is',
 'that',
 'primitives',
 'are',
 'complex',
 'even',
 'if',
 'you',
 'just',
 'want',
 'to',
 'give',
 'them',
 'a',
 'new',
 'name',
 'also',
 'the',
 'expression',
 'iintinti',
 'produces',
 'an',
 'int',
 'but',
 'what',
 'should',
 'the',
 'expression',
 'ifahrenheitintfahrenheitinti',
 'produce',
 'a',
 'unitless',
 'int',
 'a',
 'fahrenheitint']

In [29]:
df.loc[df['text'].apply(lambda x: len(str(x).split(" "))).values > 30]

0          I started to write a C++ template class that w...
1          &gt; I&#x27;m guessing over $200B of Amazon&#x...
2          I don&#x27;t know what to say - that just soun...
4          With the current trend of simplifying your int...
10         I&#x27;m sure there are many variations to thi...
                                 ...                        
1319844    I still get a small thrill from stepping onto ...
1319852    Diligent Robotics | Austin, TX | Robotics Soft...
1319854    Wait, were you an OA customer? Tell us more!<p...
1319855    At least the fear of a world destroying AI is ...
1319856    This leads to such elegant conflicts as: what ...
Name: text, Length: 654405, dtype: object

In [21]:
df['text']

0          I started to write a C++ template class that w...
1          &gt; I&#x27;m guessing over $200B of Amazon&#x...
2          I don&#x27;t know what to say - that just soun...
3          if you&#x27;re going to make the accusation th...
4          With the current trend of simplifying your int...
                                 ...                        
1319853    Good, get rid of duplicate products, makes sense.
1319854    Wait, were you an OA customer? Tell us more!<p...
1319855    At least the fear of a world destroying AI is ...
1319856    This leads to such elegant conflicts as: what ...
1319857                                                  NaN
Name: text, Length: 1319858, dtype: object

In [26]:
soup = BeautifulSoup(['text'][0])

In [9]:
df['text'][0].lower()

'i started to write a c++ template class that would implement strongly-typed ints (so celsius and fahrenheit types could behave like ints, but have distinct types).<p>i gave up after this "simple" idea approached 200 lines of code implementing all the operator overloads. i guess the lesson is that primitives are complex, even if you just want to give them a new name. also, the expression <i>int/int</i> produces an int, but what should the expression <i>fahrenheitint/fahrenheitint</i> produce? a unitless int? a fahrenheitint?'

In [16]:
soup.get_text()

'I started to write a C++ template class that would implement strongly-typed ints (so Celsius and Fahrenheit types could behave like ints, but have distinct types).I gave up after this "simple" idea approached 200 lines of code implementing all the operator overloads. I guess the lesson is that primitives are complex, even if you just want to give them a new name. Also, the expression int/int produces an int, but what should the expression FahrenheitInt/FahrenheitInt produce? A unitless int? A FahrenheitInt?'